# Nhận diện tin tức giả dựa trên bộ dữ liệu tin tức tiếng anh

Thêm các thư viện cần thiết

In [2]:
import pandas as pd
import numpy as np

## Xử dữ lý dữ liệu 

Đọc dữ liệu

In [4]:
data = pd.read_csv("./news.csv")
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [11]:
#Kiem tra du lieu
data.shape
data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

Từ kết quả trên , ta thấy không xuất hiện missing value trong dataframe

In [8]:
#Trich xuat cot output tu dataframe
y = data['label']
y.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [12]:
y.unique()

array(['FAKE', 'REAL'], dtype=object)

Ta có thể thấy dữ liệu đầu ra bao gồm 2 giá trị FAKE và REAL
- FAKE là tin tức được đánh nhãn là giả
- REAL là tin tức đúng sự thật

Ta chuyển đổi FAKE, REAL thành dữ liệu 0 và 1
- 0 đặc trưng cho FAKE
- 1 đặc trưng cho REAL

In [18]:
y = y.apply(lambda data: 0 if data == 'FAKE' else 1)

In [26]:
y.head()

0    0
1    0
2    1
3    0
4    1
Name: label, dtype: int64

Trích xuất dữ liệu cần thiết cho việc train model 

In [27]:
x = data['text']
x.head()

0    Daniel Greenfield, a Shillman Journalism Fello...
1    Google Pinterest Digg Linkedin Reddit Stumbleu...
2    U.S. Secretary of State John F. Kerry said Mon...
3    — Kaydee King (@KaydeeKing) November 9, 2016 T...
4    It's primary day in New York and front-runners...
Name: text, dtype: object

#### Chia dữ liệu thành 2 tập train và test

In [28]:
#Them thu vien ho tro viec chia du lieu
from sklearn.model_selection import train_test_split

In [48]:
#Chia tap du lieu
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.3 , random_state = 40)

In [37]:
#kiem tra tap du lieu
x_train.head()

1528    Pope Francis, in the first-ever papal address ...
5991      Crooked Hillary Campaign Used A Green Screen...
293     Lifting weights could ward off dementia and ma...
2737    The recent formal entries into the Republican ...
4258    FBI Getting Warmer: Pay-to-Play October 31, 20...
Name: text, dtype: object

In [39]:
x_test.head()

6299    Something incredible happened at the front-lin...
1816    President Obama invoked his “Yes we can” 2008 ...
4145    Sen. Cory Booker acknowledges that the Senate'...
3680    It seems like a million years ago when Democra...
3844    CHARLESTON, S.C. — Dylann Roof, who police say...
Name: text, dtype: object

#### Vector hoá ký tự thành ma trận

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

Chuyển đổi ký tự sang vector bằng Tf-Idf tuy nhiên ta thấy có một vài từ ngữ trong tiếng anh xuất hiện khá nhiều lần (this, is , are , ...) nhưng nó không ảnh hưởng đến việc quyết định kết quả có phải là fake news hay không nên ta có thể loại bỏ những từ ngữ thông dụng đó bằng stop_words

In [54]:
vectorlization = TfidfVectorizer(stop_words = 'english', max_df=0.7)

Tạo ra tập dữ liệu mới cho training

In [76]:
tfidf_train = vectorlization.fit_transform(x_train) 
tfidf_train.shape

(4434, 57457)

In [108]:
tfidf_test = vectorlization.transform(x_test) 
tfidf_test.shape

(1901, 57457)

### Traing dữ liệu

Sử dụng thuật toán phân loại hồi quy Logistic từ thư viện sklearn

In [67]:
from sklearn.linear_model import LogisticRegression

In [72]:
LG = LogisticRegression()
LG.fit(tfidf_train,y_train)

LogisticRegression()

In [98]:
y_pred_lg = LG.predict(tfidf_test)

Thuật toán Decision Tree

In [89]:
from sklearn import tree

In [92]:
TCls = tree.DecisionTreeClassifier()
TCls = TCls.fit(tfidf_train,y_train)

In [93]:
y_pre_tree = TCls.predict(tfidf_test)

Thuật toán Passive Aggression Classifier 

In [101]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [104]:
pac=PassiveAggressiveClassifier()
pac.fit(tfidf_train,y_train)

PassiveAggressiveClassifier()

In [105]:
y_pre_pac = pac.predict(tfidf_test)

### Đánh giá dữ liệu đầu ra 

In [107]:
#Thu vien ho tro danh gia du lieu dau ra
from sklearn.metrics import accuracy_score

Dữ liệu được training bằng thuật toán hồi quy Logistic

In [99]:
accuracy_score(y_test, y_pred_lg)

0.9132035770647028

Dữ liệu được training bằng thuật toán Decision Tree

In [100]:
accuracy_score(y_test, y_pre_tree)

0.7958968963703315

Dữ liệu được training bằng thuật toán Passive Aggression Classifier 

In [106]:
accuracy_score(y_test, y_pre_pac)

0.9342451341399264

### Kết luận

Từ trên ta thấy dữ liệu được traing bằng thuật toán Passive Aggression Classifier và Logistic Regression cho ra kết quả khá tốt (93,42% và 91.32%)